<a href="https://colab.research.google.com/github/Jushef/Azure-AutoML/blob/main/AutoML%20Jupyter%20Notebook/Diabetes_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cargar librerías Azure
from azureml.core import Workspace, Dataset, Datastore, Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails

# Cargar Bibliotecas No-Azure
import pandas as pd
import numpy as np

In [ ]:
# Compruebe la versión base de AzureMl
import azureml.core
print("You are currently using version " + azureml.core.VERSION + " of the Azure ML SDK")

You are currently using version 1.37.0 of the Azure ML SDK


In [ ]:
# Recupere su espacio de trabajo por nombre rellenando los valores en minúsculas entre comillas dobles
ws = Workspace.from_config()

In [ ]:
# Recuperar el objetivo informático para ejecutar AutoML de forma remota
compute_name = 'automl-compute'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
# Recupere su Datastore por nombre rellenando los valores en minúsculas entre comillas dobles
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
# Si ya ha creado el conjunto de datos de la diabetes, recupérelo
dataset_name = "Diabetes Sample"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')

In [ ]:
# Si aún no ha hecho el conjunto de datos de la diabetes, regístrelo

# importar el conjunto de datos abiertos sobre la diabetes
from azureml.opendatasets import Diabetes

# Recuperar el conjunto de datos sobre la diabetes
diabetes = Diabetes.get_tabular_dataset()

# Registrar el conjunto de datos
# Establezca el nombre del conjunto de datos, la descripción y las etiquetas como desee
data_registered = diabetes.register(workspace=ws,
                                              name='Diabetes Sample', 
                                              description='Sample Diabetes Data from Azure Open Datasets', 
                                              tags= {'fileType': 'Tabular', "project" : "Diabetes"}, 
                                              create_new_version=True)

In [ ]:
# Vea su conjunto de datos convirtiéndolo en pandas
dataset.take(10).to_pandas_dataframe()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.00,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.00,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.00,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.00,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.00,4.2905,80,135
5,23,1,22.6,89.0,139,64.8,61.0,2.00,4.1897,68,97
6,36,2,22.0,90.0,160,99.6,50.0,3.00,3.9512,82,138
7,66,2,26.2,114.0,255,185.0,56.0,4.55,4.2485,92,63
8,60,2,32.1,83.0,179,119.4,42.0,4.00,4.4773,94,110
9,29,1,30.0,85.0,180,93.4,43.0,4.00,5.3845,88,310


In [ ]:
# Convierta su conjunto de datos en un marco de datos de Pandas
dfRaw = dataset.to_pandas_dataframe()

# Eliminar las columnas de edad y sexo
df = dfRaw.drop(['AGE','SEX'], axis=1)

In [ ]:
# Registre un nuevo conjunto de datos utilizando el marco de datos con las columnas Edad y Sexo eliminadas
Dataset.Tabular.register_pandas_dataframe(df, datastore,
                                          "Diabetes Sample Age/Sex Dropped")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/5423c6ab-d1b2-4310-abb7-5546cd676b78/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/5423c6ab-d1b2-4310-abb7-5546cd676b78/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "f47ae117-ab82-48bf-833d-634c85097c30",
    "name": "Diabetes Sample Age/Sex Dropped",
    "version": 1,
    "workspace": "Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group')"
  }
}

In [ ]:
# Cree un nombre para su Experimento de Regresión de Diabetes AutoML
experiment_name = 'Diabetes-Sample-Regression'

# Establezca el experimento en su espacio de trabajo
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# Establezca las variables para su ejecución de AutoML
target_column = 'Y'
task = 'regression'
primary_metric = 'normalized_root_mean_squared_error'
featurization = 'auto'

In [ ]:
# Establezca su configuración AutoML
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=5,
                     model_explainability=True)

In [ ]:
# Entrene su modelo de regresión de diabetes AutoML
AutoML_run = exp.submit(config, show_output = True)
RunDetails(AutoML_run).show()

Submitting remote run.
No run_configuration provided, running on automl-compute with default configuration
Running on remote compute: automl-compute


Experiment,Id,Type,Status,Details Page,Docs Page
Diabetes-Sample-Regression,AutoML_c37a892e-9db6-4141-bedf-6a4f88439f65,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

*********************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Registre su modelo AutoML entrenado en su espacio de trabajo AMLS
description = 'Best AutoML Regression Run using Diabetes Sample Data. This model requires the Age and Sex Columns.' 
tags = {'project' : "Diabetes", "creator" : "your name"} 
model_name = 'Diabetes-AllData-Regression-AutoML' 

AutoML_run.register_model(model_name=model_name,description=description,tags=tags)

Model(workspace=Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group'), name=Diabetes-AllData-Regression-AutoML, id=Diabetes-AllData-Regression-AutoML:1, version=1, tags={'project': 'Diabetes', 'creator': 'your name'}, properties={})

In [ ]:
# Registre su modelo AutoML entrenado en su espacio de trabajo AMLS utilizando la puntuación R2 en lugar del error medio cuadrático normalizado
description = 'Best AutoML Regression Run using Diabetes Sample Data. This model requires the Age and Sex Columns.' 
tags = {'project' : "Diabetes", "creator" : "your name", "metric" : "R2"} 
model_name = 'Diabetes-AllData-Regression-AutoML-R2' 
AutoML_run.register_model(model_name=model_name, description=description, tags=tags, metric = 'r2_score')

Model(workspace=Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group'), name=Diabetes-AllData-Regression-AutoML-R2, id=Diabetes-AllData-Regression-AutoML-R2:1, version=1, tags={'project': 'Diabetes', 'creator': 'your name', 'metric': 'R2'}, properties={})